# Outcome oriented prediction with CNN

In [57]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.utils as torch_utils
import pandas as pd
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

os.environ["CUDA_VISIBLE_DEVICES"]="0"
!nvidia-smi

Mon Jul 27 17:26:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.51                 Driver Version: 396.51                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0  On |                  N/A |
| 25%   38C    P8    16W / 250W |   2505MiB / 11177MiB |     27%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:03:00.0 Off |                  N/A |
| 23%   35C    P8    16W / 250W |     12MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

## Construct custom dataloader

In [62]:
class Customdataset(Dataset):
    def __init__(self,x_data,y_data,transform=None):
        '''
        Call stored dataset
        
        Params
        second: Ellapsed second from the beginning of events 
        encoding_type: Encoding method for outcomeprediction ex) Static, last_state, aggregation, etc.
        '''
        
        
        
        # Transforms
        self.transform = transform
        self.y_data=y_data.to_numpy()
        self.x_data=x_data.to_numpy()
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self,idx):
        
        # Convert x and y data to torch flaot tensor
        x = torch.FloatTensor(self.x_data[idx])
        y = self.y_data[idx]
        return x,y

In [63]:
encoding_type = 'last_state'
prefix = 5
input_data = pd.read_csv('../data/'+encoding_type+'_'+str(prefix)+'.csv')
y_data = input_data.loc[:,['Label']]
input_data = input_data.drop(['Label'],axis=1)
x_data = input_data

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.33, random_state=69)


In [65]:
trainset = Customdataset(x_train,y_train)
testset = Customdataset(x_test,y_test)

In [68]:
batch_size =5
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testset,batch_size=1)

In [ ]:
class CNN(nn.Module):
    def __init__(self, input_size,hidden_size, num_layers, output_size,batch_size):
        super(CNN,self).__init__()
        
        self.input_size =  input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        
        # GRU part
        self.gru = nn.GRU(6000,hidden_size,num_layers,batch_first=True,dropout=0.2)
        self.relu = nn.ReLU()


        # CNN part
        self.cnn1 = nn.Sequential(
            nn.Conv1d(input_size,6000,2,padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(p=0.2)
        )

        self.cnn2 = nn.Sequential(
            nn.Conv1d(6000,6000,2,padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(p=0.2)
        )

        # Linear        
        self.fcs =nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(hidden_size,500),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(500, 250),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(250, output_size),
        )
        
        
        
    def forward(self,inputs,hidden):        
        hidden = hidden.reshape(self.num_layers,batch_size,-1)        
        inputs = inputs.reshape(batch_size,-1,timepoint)
        output = self.cnn1(inputs)
        output = self.cnn2(output)
        output = output.reshape([batch_size,-1,output.shape[1]])
        
        output,hidden = self.gru(output,hidden)
        output = self.relu(output)
        output = self.fcs(output)
        output = output[:,-1]
 
        return output,hidden
    
    def initHidden(self):
        return torch.zeros(1,batch_size,self.hidden_size)